In [1]:
from hw1 import Composer
from midi2seq import process_midi_seq, seq2piano, random_piano, piano2seq, segment
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
sequence = process_midi_seq(maxlen=50,n=15000)
print(sequence.shape)

notes = np.unique(sequence)
print(f'number of unique notes are {len(notes)} notes')
print(notes)

(15115, 51)
number of unique notes are 294 notes
[ 21  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 149 151 152 153 154 155
 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191
 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227
 228 229 230 231 232 233 256 257 258 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285
 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303
 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321
 3

In [4]:
X_train = sequence[:,:-1]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(X_train)

X_train = X_train.reshape((-1,X_train.shape[1],1))



Y_train = sequence[:,-1]
Y_train = Y_train.reshape((-1,1))

X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

X_train.shape, Y_train.shape

(torch.Size([15115, 50, 1]), torch.Size([15115, 1]))

In [5]:
from torch.utils.data import Dataset
class MidiComposerDataset(Dataset):
    def __init__(self,labels, x_sequence, y_next):
        self.x_sequence = x_sequence
        self.y_next = y_next
        self.labels = labels

    def __len__(self):
        return len(self.y_next)

    def one_hot_encode(self, labels, note):
        return torch.tensor(note == labels).float()
        
    def __getitem__(self, idx):
        action = int(self.y_next[idx][0])
        encode_action = self.one_hot_encode(self.labels, action)
        return dict(
            sequence = self.x_sequence[idx],
            action = encode_action
        )

In [6]:
train_dataset = MidiComposerDataset(notes, X_train,Y_train)

In [7]:
from torch.utils.data import DataLoader

BATCH_SIZE = 100

train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE, shuffle=True)

In [8]:
for _, batch in enumerate(train_loader):
    sequence_batch , action_batch = batch['sequence'].to(device) , batch['action'].to(device) 
    print(sequence_batch.shape, action_batch.shape)
    break

torch.Size([100, 50, 1]) torch.Size([100, 294])


In [9]:
class ComposerModel(nn.Module):
    def __init__(self, n_classes, n_input=1, n_hidden=256, n_layers=2):
        super().__init__()
        self.num_stacked_layers = n_layers
        self.hidden_size = n_hidden
        
        self.lstm = nn.LSTM(input_size=n_input, hidden_size=n_hidden, num_layers=n_layers, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        # Output layer
        self.linear = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        lstm_out, _ = self.lstm(x, (h0, c0))
        # take only the last output
        out = lstm_out[:, -1, :]
        # produce output
        out = self.linear(self.dropout(out))
        return out

In [10]:
classes = len(notes)
model = ComposerModel(classes,1,256, 2)
model.to(device)

ComposerModel(
  (lstm): LSTM(1, 256, num_layers=2, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear): Linear(in_features=256, out_features=294, bias=True)
)

In [20]:
learning_rate = 0.0001
loss_function = nn.CrossEntropyLoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0
    
    for batch_index, batch in enumerate(train_loader):
        sequence_batch , action_batch = batch['sequence'].to(device) , batch['action'].to(device)
        
        output = model(sequence_batch)
        loss = loss_function(output, action_batch)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0
    print()
torch.save(model, "composer.pth")

In [23]:
num_epochs = 1000
for epoch in range(num_epochs):
    train_one_epoch()

Epoch: 1
Batch 100, Loss: 392.799

Epoch: 2
Batch 100, Loss: 391.915

Epoch: 3
Batch 100, Loss: 386.456

Epoch: 4
Batch 100, Loss: 380.950

Epoch: 5
Batch 100, Loss: 382.062

Epoch: 6
Batch 100, Loss: 377.308

Epoch: 7
Batch 100, Loss: 374.197

Epoch: 8
Batch 100, Loss: 374.159

Epoch: 9
Batch 100, Loss: 372.619

Epoch: 10
Batch 100, Loss: 370.664

Epoch: 11
Batch 100, Loss: 367.824

Epoch: 12
Batch 100, Loss: 364.373

Epoch: 13
Batch 100, Loss: 363.905

Epoch: 14
Batch 100, Loss: 363.311

Epoch: 15
Batch 100, Loss: 360.885

Epoch: 16
Batch 100, Loss: 360.044

Epoch: 17
Batch 100, Loss: 358.365

Epoch: 18
Batch 100, Loss: 358.879

Epoch: 19
Batch 100, Loss: 357.683

Epoch: 20
Batch 100, Loss: 355.683

Epoch: 21
Batch 100, Loss: 355.230

Epoch: 22
Batch 100, Loss: 356.597

Epoch: 23
Batch 100, Loss: 354.055

Epoch: 24
Batch 100, Loss: 351.873

Epoch: 25
Batch 100, Loss: 352.364

Epoch: 26
Batch 100, Loss: 353.344

Epoch: 27
Batch 100, Loss: 350.092

Epoch: 28
Batch 100, Loss: 351.620

E

In [25]:
torch.save(model, "composer.pth")